## GWAS catalog filtering notebook

The purpose of this notebook is to filter GWAS associations and traits based on the curation comments and our needs

In [32]:
import polars as pl
from pathlib import *

### Setting up paths

In [33]:
base = Path(".")
data = base / "data"
input = data / "input"
output = data / "output"

In [34]:
traits_path = input / "traits_no_measurement.tsv"
traits_df = pl.read_csv(str(traits_path), sep="\t")
traits_df.head(3)

trait,trait_uri,studies,association_counts,papers,Olga's comments
str,str,i64,i64,i64,str
"""clinical treat...","""http://www.ebi...",687,31,687,null
"""employment sta...","""http://www.ebi...",687,10,687,null
"""cortical thick...","""http://www.ebi...",325,1292,325,null


Filter by curators comments and fix typos

In [35]:
comments = traits_df["Olga's comments"].to_list()

curator_comments = (
    pl.col("Olga's comments")
        .str.strip() # steam leading and ending spaces, they are often added accidentally and cause troubles
        .str.to_lowercase()
        .str.replace(" ", "_") # let's go dash instead of spaces
        .str.replace("/", ", ").str.replace("\?",  "") #clean up from some redundant symbols
        .str.replace("muskuloskeletal", "musculoskeletal").str.replace('musculosceletal', "musculoskeletal") # typos correction
        .str.replace("obesiy", 'obesity')
        .str.replace("homeostais","homeostasis")
        .str.replace(", ", "_and_").str.replace(",", "_and_") #let's first use end and then check if we can go further
) # cleaned up Olga's comments column

traits_filtered: pl.DataFrame = traits_df.filter(pl.col("Olga's comments").is_not_null())\
    .with_column(curator_comments).rename({"Olga's comments": "curator_comments"})
traits_filtered = traits_filtered.with_column(pl.col('trait').str.to_lowercase())
traits_filtered.head(10)


trait,trait_uri,studies,association_counts,papers,curator_comments
str,str,i64,i64,i64,str
"""type 2 diabete...","""http://purl.ob...",175,5003,175,"""glucose_homeos..."
"""body mass inde...","""http://www.ebi...",151,8912,151,"""glucose_homeos..."
"""covid-19""","""http://purl.ob...",123,659,123,"""lung"""
"""asthma""","""http://purl.ob...",112,1475,112,"""lung"""
"""alzheimer dise...","""http://purl.ob...",89,669,89,"""mental"""
"""bone fracture""","""http://www.ebi...",85,53,85,"""musculoskeleta..."
"""systolic blood...","""http://www.ebi...",73,3435,73,"""cardiovascular..."
"""coronary arter...","""http://www.ebi...",72,2408,72,"""cardiovascular..."
"""diastolic bloo...","""http://www.ebi...",68,2316,68,"""cardiovascular..."


In [36]:
traits_grouped = traits_filtered.groupby("curator_comments").agg(pl.all())

traits = traits_grouped.explode([c for c in traits_grouped.columns if c != "curator_comments"])
traits.head(10)

curator_comments,trait,trait_uri,studies,association_counts,papers
str,str,str,i64,i64,i64
"""musculoskeleta...","""bone fracture""","""http://www.ebi...",85,53,85
"""musculoskeleta...","""osteoarthritis...","""http://purl.ob...",60,99,60
"""musculoskeleta...","""bone density""","""http://www.ebi...",53,692,53
"""musculoskeleta...","""lean body mass...","""http://www.ebi...",45,523,45
"""musculoskeleta...","""osteoporosis""","""http://www.ebi...",33,45,33
"""musculoskeleta...","""heel bone mine...","""http://www.ebi...",25,4690,25
"""musculoskeleta...","""osteoarthritis...","""http://www.ebi...",21,64,21
"""musculoskeleta...","""femoral neck b...","""http://www.ebi...",17,197,17
"""musculoskeleta...","""spine bone min...","""http://www.ebi...",17,145,17


## Writing results to folders

let's clean the output first

In [37]:
import shutil
if output.exists():
    shutil.rmtree(output)
output.mkdir()

Let's create folders for modules

In [38]:
modules = traits_filtered.select([pl.col("curator_comments").unique()]).to_series().to_list()
modules

['metabolic_health_and_obesity',
 'longevity',
 'lung',
 'other',
 'musculoskeletal',
 'cardiovascular',
 'inflammation',
 'mental',
 'glucose_homeostasis_and_obesity']

In [39]:
for item in modules:
    item_folder = output / item
    item_folder.mkdir()
    file_name = item + '.tsv'
    file_path = item_folder / file_name
    print(f"writing module {item} to {item_folder} with traits file {file_path}\n")
    module_df = traits.filter(pl.col('curator_comments')==item)

    module_df.write_csv(file = str(file_path), sep='\t')


writing module metabolic_health_and_obesity to data\output\metabolic_health_and_obesity with traits file data\output\metabolic_health_and_obesity\metabolic_health_and_obesity.tsv

writing module longevity to data\output\longevity with traits file data\output\longevity\longevity.tsv

writing module lung to data\output\lung with traits file data\output\lung\lung.tsv

writing module other to data\output\other with traits file data\output\other\other.tsv

writing module musculoskeletal to data\output\musculoskeletal with traits file data\output\musculoskeletal\musculoskeletal.tsv

writing module cardiovascular to data\output\cardiovascular with traits file data\output\cardiovascular\cardiovascular.tsv

writing module inflammation to data\output\inflammation with traits file data\output\inflammation\inflammation.tsv

writing module mental to data\output\mental with traits file data\output\mental\mental.tsv

writing module glucose_homeostasis_and_obesity to data\output\glucose_homeostasis_an

dowloading GWASS catalogs to input folder

In [40]:
import urllib.request


local_associations = "gwass_associations.tsv"
path_associations = input / local_associations

local_studies = "gwass_studies.tsv"
path_studies = input / local_studies

if not path_associations.is_file():
    # Associations file v1.0.2
    associations_link = "https://www.ebi.ac.uk/gwas/api/search/downloads/alternative"
    urllib.request.urlretrieve(associations_link, path_associations)

if not path_studies.is_file():
    # Studies file v1.0.3
    studies_link = "https://www.ebi.ac.uk/gwas/api/search/downloads/studies_new"
    urllib.request.urlretrieve(studies_link, path_studies)

reading gwass catalogues and filtering needed rows

In [41]:
associations_df = pl.read_csv(str(path_associations),dtypes = {'CHR_ID': pl.Utf8,'CHR_POS': pl.Utf8, 'SNP_ID_CURRENT': pl.Utf8}, sep="\t")
associations_df = associations_df.with_column(pl.col('DISEASE/TRAIT').str.to_lowercase())
associations_df.head(3)

DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,REGION,CHR_ID,CHR_POS,REPORTED GENE(S),MAPPED_GENE,UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,UPSTREAM_GENE_DISTANCE,DOWNSTREAM_GENE_DISTANCE,STRONGEST SNP-RISK ALLELE,SNPS,MERGED,SNP_ID_CURRENT,CONTEXT,INTERGENIC,RISK ALLELE FREQUENCY,P-VALUE,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,i64,str,str,i64,str,f64,f64,str,f64,str,str,str,str,str,str,str
"""2016-01-27""",25673413,"""Locke AE""","""2015-02-12""","""Nature""","""www.ncbi.nlm.n...","""Genetic studie...","""body mass inde...","""up to 104,666 ...","""up to 48,274 E...","""5q33.2""","""5""","""154158333""","""GALNT10""","""MFAP3""",null,null,"""ENSG0000003774...",null,null,"""rs7715256-G""","""rs7715256""",0,"""7715256""","""non_coding_tra...",0,"""0.421""",0.0000002,6.69897,"""(EA)""",0.016,"""[0.01-0.022] k...","""Affymetrix, Il...","""N""","""body mass inde...","""http://www.ebi...","""GCST002783""","""Genome-wide ge..."
"""2016-01-27""",25673413,"""Locke AE""","""2015-02-12""","""Nature""","""www.ncbi.nlm.n...","""Genetic studie...","""body mass inde...","""up to 104,666 ...","""up to 48,274 E...","""8p12""","""8""","""34646258""","""UNC5D""","""RPL10AP3 - LIN...","""ENSG0000021526...","""ENSG0000025434...",null,322616,137770,"""rs7844647-T""","""rs7844647""",0,"""7844647""","""regulatory_reg...",1,"""0.739""",0.000005,5.30103,"""(EA)""",0.016,"""[0.0089-0.0227...","""Affymetrix, Il...","""N""","""body mass inde...","""http://www.ebi...","""GCST002783""","""Genome-wide ge..."
"""2016-01-27""",25673413,"""Locke AE""","""2015-02-12""","""Nature""","""www.ncbi.nlm.n...","""Genetic studie...","""body mass inde...","""up to 104,666 ...","""up to 48,274 E...","""10q23.1""","""10""","""85651147""","""GRID1""","""GRID1""",null,null,"""ENSG0000018277...",null,null,"""rs7899106-G""","""rs7899106""",0,"""7899106""","""intron_variant...",0,"""0.052""",3.0000e-8,7.522879,"""(EA)""",0.04,"""[0.026-0.053] ...","""Affymetrix, Il...","""N""","""body mass inde...","""http://www.ebi...","""GCST002783""","""Genome-wide ge..."


In [44]:
studies_df = pl.read_csv(str(path_studies), sep="\t")
studies_df = studies_df.with_column(pl.col(['MAPPED_TRAIT','DISEASE/TRAIT']).str.to_lowercase())
studies_df.head(3)

DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY,SUMMARY STATS LOCATION,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI
str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""2016-10-14""",26626624,"""Stuart PE""","""2015-11-28""","""Am J Hum Genet...","""www.ncbi.nlm.n...","""Genome-wide As...","""cutaneous psor...","""1,363 European...","""up to 2,969 Eu...","""Illumina [1153...",10,"""cutaneous psor...","""http://www.ebi...","""GCST003269""","""Genome-wide ge...",null,null,null,null,null,null
"""2016-10-14""",26626624,"""Stuart PE""","""2015-11-28""","""Am J Hum Genet...","""www.ncbi.nlm.n...","""Genome-wide As...","""psoriasis vulg...","""4,007 European...","""up to 9,075 Eu...","""Illumina [up t...",39,"""psoriasis vulg...","""http://www.ebi...","""GCST003268""","""Genome-wide ge...",null,null,null,null,null,null
"""2016-10-14""",26626624,"""Stuart PE""","""2015-11-28""","""Am J Hum Genet...","""www.ncbi.nlm.n...","""Genome-wide As...","""psoriatic arth...","""1,946 European...","""up to 2,883 Eu...","""Illumina [1153...",14,"""psoriatic arth...","""http://www.ebi...","""GCST003270""","""Genome-wide ge...",null,null,null,null,null,null


let's join tables and creat new db files

In [46]:
from pycomfort.files import *

folders = dirs(output)
for f in folders:
    file = with_ext(f, 'tsv')[0]
    df = pl.read_csv(str(file), sep='\t')

    file_prefix = file.stem
    file_name_associations = file_prefix + "_associations" + ".tsv"
    file_name_studies = file_prefix + "_studies" + ".tsv"

    path_associations_output = f / file_name_associations
    path_studies_output = f / file_name_studies

    df_studies = (
    df
    .join(
        studies_df,
        left_on="trait_uri",
        right_on="MAPPED_TRAIT_URI",
        how="inner")
    )
    df_studies.write_csv(file = str(path_studies_output), sep='\t')
    df_associations = (
    df
    .join(
        associations_df,
        left_on="trait_uri",
        right_on="MAPPED_TRAIT_URI",
        how="inner")
    )

    # filtering only big df
    if df_associations.height > 100:
        df_associations =  df_associations.filter(pl.col("P-VALUE") < 0.05)
        df_associations.write_csv(file = str(path_associations_output), sep='\t')
    else:
        df_associations.write_csv(file = str(path_associations_output), sep='\t')
